# Exploratory Data Analysis of Rotten Tomatoes Top 100 in each Genre
Rotten Tomato dataset was retrived from kaggle @ 
https://www.kaggle.com/datasets/prasertk/top-100-rotten-tomatoes-movies-by-genres

In this analysis I have tried to answer the following questions: 
1. Explore the year that had the most popular titles
2. What correlations, if any, exist?  
3. Find the most popular movies in each genre
4. The most popular movie overall

In [568]:
!pip install plotly

In [569]:
# Importing libraries 
import numpy as np 
import pandas as pd
import matplotlib. pyplot as plt 
%matplotlib inline

import seaborn as sns 
import plotly.express as px

In [570]:
rt_df = pd.read_csv('top_100_movies_by_genres.csv')

# Data wrangling 
Now I will spend some time understading the values within my dataset, cleaning as necessary, and preparing it for further evaluation. 

In [571]:
rt_df.head()

,Genre,Rank,RatingTomatometer,Title,No. of Reviews
0,Action & Adventure,1.0,96%,Black Panther (2018),525
1,Action & Adventure,2.0,94%,Avengers: Endgame (2019),547
2,Action & Adventure,3.0,97%,Mission: Impossible - Fallout (2018),437
3,Action & Adventure,4.0,97%,Mad Max: Fury Road (2015),434
4,Action & Adventure,5.0,97%,Spider-Man: Into the Spider-Verse (2018),393


In [572]:
rt_df.shape

(1612, 5)

In [573]:
rt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1612 entries, 0 to 1611
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Genre              1612 non-null   object 
 1   Rank               1612 non-null   float64
 2   RatingTomatometer  1612 non-null   object 
 3   Title              1612 non-null   object 
 4   No. of Reviews     1612 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 63.1+ KB


In [574]:
rt_df.isnull().sum()

Genre                0
Rank                 0
RatingTomatometer    0
Title                0
No. of Reviews       0
dtype: int64

In [575]:
rt_df.columns

Index(['Genre', 'Rank', 'RatingTomatometer', 'Title', 'No. of Reviews'], dtype='object')

My dataframe has 1612 rows and five columns. It has no missing values. From here, I will begin to analyze this data with descriptive statistics.


I will focus on the following columns:
1. RankingTomatometer: This dataset only includes the rating that was given to it by critics
2. No. of Reviews: The number of critics who gave their opinions on this dataset. 

In [576]:
rt_df['No. of Reviews'].min()

40

In [577]:
rt_df['No. of Reviews'].max()

571

In [578]:
rt_df['No. of Reviews'].mean()

148.0651364764268

Most movies have around 148 reviews. Now I will check my data for outliers.

In [579]:
fig1 = px.histogram(rt_df, x='No. of Reviews')
fig1.show()

This distribution is not normal, so I will be adding a new column with this information normalized through min/max scaling. 

In [580]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

rt_df[['Scaled Reviews']] = scaler.fit_transform(rt_df[["No. of Reviews"]])

rt_df.head()

,Genre,Rank,RatingTomatometer,Title,No. of Reviews,Scaled Reviews
0,Action & Adventure,1.0,96%,Black Panther (2018),525,0.913371
1,Action & Adventure,2.0,94%,Avengers: Endgame (2019),547,0.954802
2,Action & Adventure,3.0,97%,Mission: Impossible - Fallout (2018),437,0.747646
3,Action & Adventure,4.0,97%,Mad Max: Fury Road (2015),434,0.741996
4,Action & Adventure,5.0,97%,Spider-Man: Into the Spider-Verse (2018),393,0.664783


In [581]:
fig2 = px.histogram(rt_df, x="Scaled Reviews")
fig2.show()

The other column I would like to cover with some statistical analysis is classified as an 'object' type rather than an integer. I need to correct this in order to preform calculations.

In [582]:
rt_df['RatingTomatometer'] = rt_df["RatingTomatometer"].str.rstrip("%").astype('float')/100.0

In [583]:
rt_df.head()

,Genre,Rank,RatingTomatometer,Title,No. of Reviews,Scaled Reviews
0,Action & Adventure,1.0,0.96,Black Panther (2018),525,0.913371
1,Action & Adventure,2.0,0.94,Avengers: Endgame (2019),547,0.954802
2,Action & Adventure,3.0,0.97,Mission: Impossible - Fallout (2018),437,0.747646
3,Action & Adventure,4.0,0.97,Mad Max: Fury Road (2015),434,0.741996
4,Action & Adventure,5.0,0.97,Spider-Man: Into the Spider-Verse (2018),393,0.664783


In [584]:
rt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1612 entries, 0 to 1611
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Genre              1612 non-null   object 
 1   Rank               1612 non-null   float64
 2   RatingTomatometer  1612 non-null   float64
 3   Title              1612 non-null   object 
 4   No. of Reviews     1612 non-null   int64  
 5   Scaled Reviews     1612 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 75.7+ KB


In [585]:
rt_df["RatingTomatometer"].min()

0.02

In [586]:
rt_df["RatingTomatometer"].max()

1.0

In [587]:
rt_df["RatingTomatometer"].mean()

0.927059553349876

This dataset specifically focuses on the top 100 in each genre so I expect the mean to be high. 

(I want to seperate this out by genre, by movie data, and limit it down to the top 10 movies in each genre) 

In [588]:
fig3 = px.histogram(rt_df, x="RatingTomatometer")
fig3.show()

# Question 1: What year had the most popular titles?
I will start by seperating out titles and release dates

In [589]:
new = rt_df["Title"].str.split("(", n = 1, expand=True)

rt_df["Movie Title"] = new[0]
rt_df["Year"] = new[1]

rt_df.head()

,Genre,Rank,RatingTomatometer,Title,No. of Reviews,Scaled Reviews,Movie Title,Year
0,Action & Adventure,1.0,0.96,Black Panther (2018),525,0.913371,Black Panther,2018)
1,Action & Adventure,2.0,0.94,Avengers: Endgame (2019),547,0.954802,Avengers: Endgame,2019)
2,Action & Adventure,3.0,0.97,Mission: Impossible - Fallout (2018),437,0.747646,Mission: Impossible - Fallout,2018)
3,Action & Adventure,4.0,0.97,Mad Max: Fury Road (2015),434,0.741996,Mad Max: Fury Road,2015)
4,Action & Adventure,5.0,0.97,Spider-Man: Into the Spider-Verse (2018),393,0.664783,Spider-Man: Into the Spider-Verse,2018)


In [590]:
rt_df["Year"].isnull().sum()

0

In [591]:
rt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1612 entries, 0 to 1611
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Genre              1612 non-null   object 
 1   Rank               1612 non-null   float64
 2   RatingTomatometer  1612 non-null   float64
 3   Title              1612 non-null   object 
 4   No. of Reviews     1612 non-null   int64  
 5   Scaled Reviews     1612 non-null   float64
 6   Movie Title        1612 non-null   object 
 7   Year               1612 non-null   object 
dtypes: float64(3), int64(1), object(4)
memory usage: 100.9+ KB


In [592]:
rt_df["Year"].unique()

array(['2018)', '2019)', '2015)', '2017)', 'Shichinin no Samurai) (1956)',
       '1927)', '2020)', '1938)', '2016)', '1933)', '1957)', '1975)',
       '2009)', '2008)', '1948)', '1962)', '2012)', '1935)', '2011)',
       '1979)', '2014)', '1964)', '1977)', '1939)', '1971)', '1984)',
       '2021)', '1987)', '1974)', '1986)', '1980)', '2010)', '2013)',
       '2006)', '2000)', '2007)', '1951)', '1985)', '1993)',
       'Per un Pugno di Dollari) (1964)', '1989)', '2002)',
       '1984 Original) (1984)', '1959)', 'Det Sjunde inseglet) (1957)',
       '1994)', '2001)', '1956)', '1968)', '1937)', '1999)', '2003)',
       '1940)', '1995)', '1941)', '2004)', 'Ma vie de courgette) (2017)',
       '1991)', '1961)', 'Hotaru no haka) (1988)',
       'Kimi No Na Wa.) (2017)', '1988)', '2005)', '1992)', '1996)',
       'La tortue rouge) (2017)', 'Tout en haut du monde) (2016)',
       '1982)', '1942)', '1955)', 'Mononoke-hime) (1999)', '1998)',
       'O Menino e o Mundo) (2015)', 'Sennen joyû) (2

I realize that I have an error here as some titles also listed some additional information in paraenthesis after the title, which is where I seperated out the values. Since I only want numbers in this column, I will now remove some those characters. 

In [593]:
def numbers(element):
    
    return "".join(filter(str.isnumeric, element))

In [594]:
rt_df.loc[:, "YearB"] = [numbers(x) for x in rt_df.Year]
rt_df.head()

,Genre,Rank,RatingTomatometer,Title,No. of Reviews,Scaled Reviews,Movie Title,Year,YearB
0,Action & Adventure,1.0,0.96,Black Panther (2018),525,0.913371,Black Panther,2018),2018
1,Action & Adventure,2.0,0.94,Avengers: Endgame (2019),547,0.954802,Avengers: Endgame,2019),2019
2,Action & Adventure,3.0,0.97,Mission: Impossible - Fallout (2018),437,0.747646,Mission: Impossible - Fallout,2018),2018
3,Action & Adventure,4.0,0.97,Mad Max: Fury Road (2015),434,0.741996,Mad Max: Fury Road,2015),2015
4,Action & Adventure,5.0,0.97,Spider-Man: Into the Spider-Verse (2018),393,0.664783,Spider-Man: Into the Spider-Verse,2018),2018


In [595]:
rt_df["YearB"].unique()

array(['2018', '2019', '2015', '2017', '1956', '1927', '2020', '1938',
       '2016', '1933', '1957', '1975', '2009', '2008', '1948', '1962',
       '2012', '1935', '2011', '1979', '2014', '1964', '1977', '1939',
       '1971', '1984', '2021', '1987', '1974', '1986', '1980', '2010',
       '2013', '2006', '2000', '2007', '1951', '1985', '1993', '1989',
       '2002', '19841984', '1959', '1994', '2001', '1968', '1937', '1999',
       '2003', '1940', '1995', '1941', '2004', '1991', '1961', '1988',
       '2005', '1992', '1996', '1982', '1942', '1955', '1998', '1920',
       '1922', '1967', '1931', '1925', '1969', '1953', '1946', '1960',
       '1928', '1966', '1970', '1972', '1202017', '1963', '1930', '1973',
       '1954', '1958', '1934', '1936', '1950', '1949', '1952', '1921',
       '1944', '1943', '1932', '1945', '1947', '1965', '1978', '1981',
       '1997', '1976', '1990', '1983', '1929', '1742003'], dtype=object)

In [596]:
rt_df["YearB"].isnull().sum()

0

In [597]:
rt_df.drop(["Year", "Title"], axis=1, inplace=True)

In [598]:
rt_df.head()

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB
0,Action & Adventure,1.0,0.96,525,0.913371,Black Panther,2018
1,Action & Adventure,2.0,0.94,547,0.954802,Avengers: Endgame,2019
2,Action & Adventure,3.0,0.97,437,0.747646,Mission: Impossible - Fallout,2018
3,Action & Adventure,4.0,0.97,434,0.741996,Mad Max: Fury Road,2015
4,Action & Adventure,5.0,0.97,393,0.664783,Spider-Man: Into the Spider-Verse,2018


I still have some values that don't make sense in my unique list above so I'm going to double check those values now.

In [599]:
rt_df.loc[rt_df["YearB"] == "19841984"]

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB
90,Action & Adventure,91.0,0.97,73,0.062147,Ghostbusters,19841984
483,Comedy,84.0,0.97,73,0.062147,Ghostbusters,19841984
1282,Science Fiction & Fantasy,83.0,0.97,73,0.062147,Ghostbusters,19841984


In [600]:
rt_df.at[90, "YearB"]="1984"
rt_df.at[483, "YearB"]="1984"
rt_df.at[1282, "YearB"]="1984"

In [601]:
rt_df.loc[rt_df["YearB"] == "1202017"]

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB
229,Art House & International,30.0,0.99,135,0.178908,BPM,1202017


In [602]:
rt_df.at[229, "YearB"]="2017"

In [603]:
rt_df.loc[rt_df["YearB"] == "1742003"]

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB
1381,Special Interest,82.0,0.96,79,0.073446,Bus 174,1742003


In [604]:
rt_df.at[1381, "YearB"]="2003"

In [605]:
rt_df["YearB"].unique()

array(['2018', '2019', '2015', '2017', '1956', '1927', '2020', '1938',
       '2016', '1933', '1957', '1975', '2009', '2008', '1948', '1962',
       '2012', '1935', '2011', '1979', '2014', '1964', '1977', '1939',
       '1971', '1984', '2021', '1987', '1974', '1986', '1980', '2010',
       '2013', '2006', '2000', '2007', '1951', '1985', '1993', '1989',
       '2002', '1959', '1994', '2001', '1968', '1937', '1999', '2003',
       '1940', '1995', '1941', '2004', '1991', '1961', '1988', '2005',
       '1992', '1996', '1982', '1942', '1955', '1998', '1920', '1922',
       '1967', '1931', '1925', '1969', '1953', '1946', '1960', '1928',
       '1966', '1970', '1972', '1963', '1930', '1973', '1954', '1958',
       '1934', '1936', '1950', '1949', '1952', '1921', '1944', '1943',
       '1932', '1945', '1947', '1965', '1978', '1981', '1997', '1976',
       '1990', '1983', '1929'], dtype=object)

In [606]:
rt_df.sort_values("YearB")

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB
702,Horror,3.0,0.99,68,0.052731,The Cabinet of Dr. Caligari,1920
201,Art House & International,2.0,0.99,68,0.052731,The Cabinet of Dr. Caligari,1920
307,Classics,8.0,0.99,68,0.052731,The Cabinet of Dr. Caligari,1920
1006,Mystery & Suspense,7.0,0.99,68,0.052731,The Cabinet of Dr. Caligari,1920
415,Comedy,16.0,1.00,49,0.016949,The Kid,1921
...,...,...,...,...,...,...,...
1256,Science Fiction & Fantasy,57.0,0.94,283,0.457627,Raya and the Last Dragon,2021
665,Drama,66.0,0.93,420,0.715631,Nomadland,2021
753,Horror,54.0,0.98,49,0.016949,My Heart Can't Beat Unless You Tell It To,2021
536,Documentary,37.0,0.98,126,0.161959,The Velvet Underground,2021


In [607]:
fig4 = px.histogram(rt_df, x="YearB")
fig4.show()

In [608]:
rt_df["YearB"].value_counts()

2018    83
2017    80
2019    67
2020    65
2014    65
        ..
1921     2
1936     2
1928     2
1983     1
1929     1
Name: YearB, Length: 99, dtype: int64

In [609]:
rt_df["YearB"].max()

'2021'

The year with the most popular titles was 2018

# Seperate out dataframe by Specific Criteria
I'm seperating out my dataframe by the following criteria to make further analysis simpler. 
1. Seperate dataframe by genre
2. Seperate dataframe by decade 

In [610]:
rt_df["YearB"].min()

'1920'

In [611]:
# I had an error here and had to strip the whitespace from my Year column in order to type the data as 'int'
rt_df["YearB"].isnull().values.any()

False

In [612]:
rt_df["YearB"] = pd.to_numeric(rt_df["YearB"].astype(str).str.strip(), errors='coerce')

In [613]:
print (rt_df["YearB"].tolist()[:20])

[2018, 2019, 2018, 2015, 2018, 2017, 2017, 2017, 2017, 2017, 2017, 1956, 2015, 1927, 2020, 1938, 2019, 2018, 2016, 1933]


In [614]:
rt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1612 entries, 0 to 1611
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Genre              1612 non-null   object 
 1   Rank               1612 non-null   float64
 2   RatingTomatometer  1612 non-null   float64
 3   No. of Reviews     1612 non-null   int64  
 4   Scaled Reviews     1612 non-null   float64
 5   Movie Title        1612 non-null   object 
 6   YearB              1612 non-null   int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 88.3+ KB


In [615]:
conditions = [
    (rt_df["YearB"] <= 1929), 
    (rt_df["YearB"] >= 1930) & (rt_df["YearB"] <= 1939), 
    (rt_df["YearB"] >= 1940) & (rt_df["YearB"] <= 1949),
    (rt_df["YearB"] >= 1950) & (rt_df["YearB"] <= 1959),
    (rt_df["YearB"] >= 1960) & (rt_df["YearB"] <= 1969),
    (rt_df["YearB"] >= 1970) & (rt_df["YearB"] <= 1979),
    (rt_df["YearB"] >= 1980) & (rt_df["YearB"] <= 1989),
    (rt_df["YearB"] >= 1990) & (rt_df["YearB"] <= 1999),
    (rt_df["YearB"] >= 2000) & (rt_df["YearB"] <= 2009),
    (rt_df["YearB"] >= 2010) & (rt_df["YearB"] <= 2019),
    (rt_df["YearB"] >= 2020) & (rt_df["YearB"] <= 2022)
]

values = ["1920s", "1930s", "1940s", "1950s", "1960s", "1970s", "1980s", "1990s", "2000s", "2010s", "2020s"]

rt_df["Decade"] = np.select(conditions, values)

rt_df.head()

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
0,Action & Adventure,1.0,0.96,525,0.913371,Black Panther,2018,2010s
1,Action & Adventure,2.0,0.94,547,0.954802,Avengers: Endgame,2019,2010s
2,Action & Adventure,3.0,0.97,437,0.747646,Mission: Impossible - Fallout,2018,2010s
3,Action & Adventure,4.0,0.97,434,0.741996,Mad Max: Fury Road,2015,2010s
4,Action & Adventure,5.0,0.97,393,0.664783,Spider-Man: Into the Spider-Verse,2018,2010s


In [616]:
rt_df["Genre"].unique()

array(['Action & Adventure', 'Animation', 'Art House & International',
       'Classics', 'Comedy', 'Documentary', 'Drama', 'Horror',
       'Kids & Family', 'Musical & Performing Arts', 'Mystery & Suspense',
       'Romance', 'Science Fiction & Fantasy', 'Special Interest',
       'Sports & Fitness', 'Television', 'Western'], dtype=object)

In [617]:
action_df = rt_df.loc[rt_df["Genre"] == "Action & Adventure"]

In [618]:
action_df.head()

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
0,Action & Adventure,1.0,0.96,525,0.913371,Black Panther,2018,2010s
1,Action & Adventure,2.0,0.94,547,0.954802,Avengers: Endgame,2019,2010s
2,Action & Adventure,3.0,0.97,437,0.747646,Mission: Impossible - Fallout,2018,2010s
3,Action & Adventure,4.0,0.97,434,0.741996,Mad Max: Fury Road,2015,2010s
4,Action & Adventure,5.0,0.97,393,0.664783,Spider-Man: Into the Spider-Verse,2018,2010s


In [619]:
animation_df = rt_df.loc[rt_df["Genre"] == "Animation"]
art_house_df = rt_df.loc[rt_df["Genre"] == "Art House & International"]
classics_df = rt_df.loc[rt_df["Genre"] == "Classics"]
comedy_df = rt_df.loc[rt_df["Genre"] == "Documentary"]
drama_df = rt_df.loc[rt_df["Genre"] == "Drama"]
doc_df = rt_df.loc[rt_df["Genre"] == "Documentary"]
horror_df = rt_df.loc[rt_df["Genre"] == "Horror"]
kids_df = rt_df.loc[rt_df["Genre"] == "Kids & Family"]
musical_df = rt_df.loc[rt_df["Genre"] == "Musical & Performing Arts"]
mystery_df = rt_df.loc[rt_df["Genre"] == "Mystery & Suspense"]
romance_df = rt_df.loc[rt_df["Genre"] == "Romance"]
scify_df = rt_df.loc[rt_df["Genre"] == "Science Fiction & Fantasy"]
special_df = rt_df.loc[rt_df["Genre"] == "Special Interest"]
sports_df = rt_df.loc[rt_df["Genre"] == "Sports & Fitness"]
tv_df = rt_df.loc[rt_df["Genre"] == "Television"]
western_df = rt_df.loc[rt_df["Genre"] == "Western"]

In [620]:
fig8 = make_subplots(
    rows=5, cols=4,
    subplot_titles=("Action", "Animation", "Art House & Intern.", "Classics", "Comedy", "Drama", "Documentary" "Horror", "Kids", "Musicals", "Mystery", "Romance", "Scify", "Special", "Sports", "Tv", "Western"))

fig8.add_trace(go.Histogram(x=action_df["YearB"]),
              row=1, col=1)

fig8.add_trace(go.Histogram(x=animation_df["YearB"]),
              row=1, col=2)

fig8.add_trace(go.Histogram(x=art_house_df["YearB"]),
              row=1, col=3)

fig8.add_trace(go.Histogram(x=classics_df["YearB"]),
              row=1, col=4)

fig8.add_trace(go.Histogram(x=comedy_df["YearB"]),
              row=2, col=1)

fig8.add_trace(go.Histogram(x=drama_df["YearB"]),
              row=2, col=2)

fig8.add_trace(go.Histogram(x=doc_df["YearB"]), 
              row=2, col=3)

fig8.add_trace(go.Histogram(x=horror_df["YearB"]),
              row=2, col=4)

fig8.add_trace(go.Histogram(x=kids_df["YearB"]),
              row=3, col=1)

fig8.add_trace(go.Histogram(x=musical_df["YearB"]),
              row=3, col=2)

fig8.add_trace(go.Histogram(x=mystery_df["YearB"]),
              row=3, col=3)

fig8.add_trace(go.Histogram(x=romance_df["YearB"]),
              row=3, col=4)

fig8.add_trace(go.Histogram(x=scify_df["YearB"]),
              row=4, col=1)

fig8.add_trace(go.Histogram(x=special_df["YearB"]),
              row=4, col=2)

fig8.add_trace(go.Histogram(x=sports_df["YearB"]),
              row=4, col=3)

fig8.add_trace(go.Histogram(x=tv_df["YearB"]),
              row=4, col=4)

fig8.add_trace(go.Histogram(x=western_df["YearB"]),
              row=5, col=1)

fig8.show()

In [621]:
early_twenties_df = rt_df.loc[rt_df["YearB"] <= 1929]

In [622]:
fig5 = px.histogram(early_twenties_df, x="YearB")
fig5.show()

Now I know my seperation by decade has worked, so I will complete the rest of the seperations in the cell below. 

In [623]:
thirties_df = rt_df.loc[(rt_df["YearB"] >= 1930) & (rt_df["YearB"] <= 1939)]
fourties_df = rt_df.loc[(rt_df["YearB"] >= 1940) & (rt_df["YearB"] <= 1949)]
fifties_df = rt_df.loc[(rt_df["YearB"] >= 1950) & (rt_df["YearB"] <= 1959)]
sixties_df = rt_df.loc[(rt_df["YearB"] >= 1960) & (rt_df["YearB"] <= 1969)]
seventies_df = rt_df.loc[(rt_df["YearB"] >= 1970) & (rt_df["YearB"] <= 1979)]
eighties_df = rt_df.loc[(rt_df["YearB"] >= 1980) & (rt_df["YearB"] <= 1989)]
ninties_df = rt_df.loc[(rt_df["YearB"] >= 1990) & (rt_df["YearB"] <= 1999)]
thousands_df = rt_df.loc[(rt_df["YearB"] >= 2000) & (rt_df["YearB"] <= 2009)]
twentyteens_df = rt_df.loc[(rt_df["YearB"] >= 2010) & (rt_df["YearB"] <= 2019)]
twentytwenties_df = rt_df.loc[(rt_df["YearB"] >= 2020) & (rt_df["YearB"] <= 2022)]

I will also add these decades as a column on my dataframe in order to utilize it later on. 

In [624]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [625]:
thousands_df.head()

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
25,Action & Adventure,26.0,0.98,298,0.485876,Up,2009,2000s
27,Action & Adventure,28.0,0.94,345,0.574388,The Dark Knight,2008,2000s
37,Action & Adventure,38.0,0.97,289,0.468927,The Hurt Locker,2009,2000s
40,Action & Adventure,41.0,0.94,355,0.593220,Star Trek,2009,2000s
52,Action & Adventure,53.0,0.95,260,0.414313,WALL-E,2008,2000s


In [626]:
fig7 = make_subplots(
    rows=4, cols=3,
    subplot_titles=("1920s", "1930s", "1940s", "1950s", "1960s", "1970s", "1980s", "1990s", "2000s", "2010s", "2020s", "Every Year"))

fig7.add_trace(go.Histogram(x=early_twenties_df["YearB"]),
              row=1, col=1)

fig7.add_trace(go.Histogram(x=thirties_df["YearB"]),
              row=1, col=2)

fig7.add_trace(go.Histogram(x=fourties_df["YearB"]),
              row=1, col=3)

fig7.add_trace(go.Histogram(x=fifties_df["YearB"]),
              row=2, col=1)

fig7.add_trace(go.Histogram(x=sixties_df["YearB"]),
              row=2, col=2)

fig7.add_trace(go.Histogram(x=seventies_df["YearB"]),
              row=2, col=3)

fig7.add_trace(go.Histogram(x=eighties_df["YearB"]),
              row=3, col=1)

fig7.add_trace(go.Histogram(x=ninties_df["YearB"]),
              row=3, col=2)

fig7.add_trace(go.Histogram(x=thousands_df["YearB"]),
              row=3, col=3)

fig7.add_trace(go.Histogram(x=twentyteens_df["YearB"]),
              row=4, col=1)

fig7.add_trace(go.Histogram(x=twentytwenties_df["YearB"]),
              row=4, col=2)

fig7.add_trace(go.Histogram(x=rt_df["YearB"]),
              row=4, col=3)

fig7.show()

#  Answer Questions
Now that I have wrangled, organized, and explored my data I will being to answer my questions. 

1. Explore the year that had the most popular titles
2. Compare the data to see if any correlation exists
3. Find the most popular movies in each genre
4. The most popular movie overall

# 1. Which year had the most popular titles?
Answer: 2018

In [627]:
rt_df["YearB"].value_counts()

2018    83
2017    80
2019    67
2020    65
2014    65
        ..
1921     2
1936     2
1928     2
1983     1
1929     1
Name: YearB, Length: 99, dtype: int64

In [628]:
fig9 = px.histogram(rt_df, x="YearB")
fig9.show()

In [629]:
fig10 = px.histogram(twentyteens_df, x="YearB")
fig10.show()

# 2.What correlations, if any, exist?
Answer: Number of Reviews and Year released have a slight positive relationship.
No onther clear correlations exist in this dataset.

In [630]:
rt_df.corr()

,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,YearB
Rank,1.000000,-0.172687,-0.205167,-0.205167,0.095446
RatingTomatometer,-0.172687,1.000000,-0.039517,-0.039517,-0.190584
No. of Reviews,-0.205167,-0.039517,1.000000,1.000000,0.537173
Scaled Reviews,-0.205167,-0.039517,1.000000,1.000000,0.537173
YearB,0.095446,-0.190584,0.537173,0.537173,1.000000


Correlations varry from -1 to 1. 1 means there is a perfect relationship. 
(notice that this only happens when the matrix crosses itself)

The closer to zero we get (either negative or positive) the weaker the 
relationship between both variables. 

In this case, it seems that the Number of Reviews and Year of release have a somewhat moderate relationship. 

Correlation is not causation, but this could be worth exploring further. 

In [631]:
fig11 = px.scatter(rt_df, x="RatingTomatometer", y="No. of Reviews")
fig11.show()

In [632]:
fig12 = px.scatter(rt_df, x="RatingTomatometer", y="No. of Reviews", color="Genre")
fig12.show()

Adding genre does not seem to create a more linear relationship, but I want to see this addition a little closer up to make sure that I haven't missed anything. 

In [633]:
fig13 = make_subplots(
    rows=5, cols=4,
    subplot_titles=("Action", "Animation", "Art House & Intern.", "Classics", "Comedy", "Drama", "Docs" "Horror", "Kids", "Musicals", "Mystery", "Romance", "Scify", "Special", "Sports", "Tv", "Western"))

fig13.add_trace(go.Scatter(x=action_df["RatingTomatometer"], y=action_df["No. of Reviews"]),
              row=1, col=1)

fig13.add_trace(go.Scatter(x=animation_df["RatingTomatometer"], y=animation_df["No. of Reviews"]),
              row=1, col=2)

fig13.add_trace(go.Scatter(x=art_house_df["RatingTomatometer"], y=art_house_df["No. of Reviews"]),
              row=1, col=3)

fig13.add_trace(go.Scatter(x=classics_df["RatingTomatometer"], y=classics_df["No. of Reviews"]),
              row=1, col=4)

fig13.add_trace(go.Scatter(x=comedy_df["RatingTomatometer"], y=comedy_df["No. of Reviews"]),
              row=2, col=1)

fig13.add_trace(go.Scatter(x=drama_df["RatingTomatometer"], y=drama_df["No. of Reviews"]),
              row=2, col=2)

fig13.add_trace(go.Scatter(x=doc_df["RatingTomatometer"], y=doc_df["No. of Reviews"]),
               row=2, col=3)

fig13.add_trace(go.Scatter(x=horror_df["RatingTomatometer"], y=horror_df["No. of Reviews"]),
              row=2, col=4)

fig13.add_trace(go.Scatter(x=kids_df["RatingTomatometer"], y=kids_df["No. of Reviews"]),
              row=3, col=1)

fig13.add_trace(go.Scatter(x=musical_df["RatingTomatometer"], y=musical_df["No. of Reviews"]),
              row=3, col=2)

fig13.add_trace(go.Scatter(x=mystery_df["RatingTomatometer"], y=mystery_df["No. of Reviews"]),
              row=3, col=3)

fig13.add_trace(go.Scatter(x=romance_df["RatingTomatometer"], y=romance_df["No. of Reviews"]),
              row=3, col=4)

fig13.add_trace(go.Scatter(x=scify_df["RatingTomatometer"], y=scify_df["No. of Reviews"]),
              row=4, col=1)

fig13.add_trace(go.Scatter(x=special_df["RatingTomatometer"], y=special_df["No. of Reviews"]),
              row=4, col=2)

fig13.add_trace(go.Scatter(x=sports_df["RatingTomatometer"], y=sports_df["No. of Reviews"]),
              row=4, col=3)

fig13.add_trace(go.Scatter(x=tv_df["RatingTomatometer"], y=tv_df["No. of Reviews"]),
              row=4, col=4)

fig13.add_trace(go.Scatter(x=western_df["RatingTomatometer"], y=western_df["No. of Reviews"]),
              row=5, col=1)

fig13.show()

In [634]:
fig14 = px.scatter(rt_df, x="RatingTomatometer", y="No. of Reviews", facet_col="Genre", facet_col_wrap=2, width=800, height=2400)
fig14.update_layout(title="Ratings vs Number of Reviews")
fig14.show()

What I gather from these relationships (or lack thereof), is that Sports & Fitness, Television, and Western are less popular genres than the others and the movies with fewer reviewers were added to the top '100'. 

In [635]:
!pip install statsmodels

In [636]:
fig15 = px.scatter(rt_df, x="YearB", y="No. of Reviews", color="Decade", trendline="lowess", trendline_scope="overall", trendline_color_override="black")
fig15.show()

In [637]:
fig16 = px.scatter(rt_df, x="YearB", y="No. of Reviews", facet_col= "Genre", facet_col_wrap= 3, trendline="ols", height=1200)
fig16.show()

Not all of these lines fit perfectly, but it does seem that the newer the release date, the more likely it is that the Number of Reviews will also be higher. This makes sense as the entertainment industry continues to grow. 

# Question 3: Find the most popular movies in each genre
Answer: 
Action & Adventure: Black Panther (2018)	
Animation: Toy Story 4	(2019)	
Art House & International: Parasite	(2019)	
Classics: It Happened One Night	(1934)	
Documentary: Won't You Be My Neighbor?	(2018)
Drama: Citizen Kane	(1941)	
Horror: Us	(2019)	
Kids & Family: The Wizard of Oz	(1939)	
Musical & Performing Arts: The Wizard of Oz	(1939)
Mystery & Suspense: Citizen Kane (1941)	
Romance: It Happened One Night	(1934)
Science Fiction & Fantasy: The Wizard of Oz	(1939)	
Special Interest: The Bride of Frankenstein	(1935)	
Sports & Fitness: Murderball (2005)
Television: Life Itself	(2014)	
Western: The Treasure of the Sierra Madre (1948)

In [638]:
fig6 = px.scatter(rt_df, x="RatingTomatometer", y="No. of Reviews", color="Genre")
fig6.show()

In [639]:
rt_df.sort_values("RatingTomatometer", ascending=False).head()

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
543,Documentary,44.0,1.0,72,0.060264,Deliver Us from Evil,2006,2000s
327,Classics,28.0,1.0,65,0.047081,Laura,1944,1940s
574,Documentary,75.0,1.0,41,0.001883,Changing the Game,2019,2010s
573,Documentary,74.0,1.0,55,0.028249,Last Train Home,2010,2010s
572,Documentary,73.0,1.0,42,0.003766,My Journey Through French Cinema,2017,2010s


In [640]:
rt_df.sort_values("No. of Reviews", ascending=False).head()

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
414,Comedy,15.0,0.85,571,1.000000,Once Upon a Time In Hollywood,2019,2010s
646,Drama,47.0,0.85,571,1.000000,Once Upon a Time In Hollywood,2019,2010s
700,Horror,1.0,0.93,553,0.966102,Us,2019,2010s
1002,Mystery & Suspense,3.0,0.93,553,0.966102,Us,2019,2010s
603,Drama,4.0,0.94,547,0.954802,Avengers: Endgame,2019,2010s


Rotten Tomatoes has some movies listed under several genres, I'll be sure to address that below.

In [641]:
# I am going to remind myself what genres I will be evaluating 
rt_df["Genre"].unique()

array(['Action & Adventure', 'Animation', 'Art House & International',
       'Classics', 'Comedy', 'Documentary', 'Drama', 'Horror',
       'Kids & Family', 'Musical & Performing Arts', 'Mystery & Suspense',
       'Romance', 'Science Fiction & Fantasy', 'Special Interest',
       'Sports & Fitness', 'Television', 'Western'], dtype=object)

In [642]:
top_action = action_df.sort_values("Rank").head(10)
top_animation = animation_df.sort_values("Rank").head(10)
top_art = art_house_df.sort_values("Rank").head(10)
top_classics = classics_df.sort_values("Rank").head(10)
top_comedy = comedy_df.sort_values("Rank").head(10)
top_doc = doc_df.sort_values("Rank").head(10)
top_drama = drama_df.sort_values("Rank").head(10)
top_horror = horror_df.sort_values("Rank").head(10)
top_kids = kids_df.sort_values("Rank").head(10)
top_musical = musical_df.sort_values("Rank").head(10)
top_mystery = mystery_df.sort_values("Rank").head(10)
top_romance = romance_df.sort_values("Rank").head(10)
top_science = scify_df.sort_values("Rank").head(10)
top_special = special_df.sort_values("Rank").head(10)
top_sports = sports_df.sort_values("Rank").head(10)
top_tv = tv_df.sort_values("Rank").head(10)
top_west = western_df.sort_values("Rank").head(10)

In [643]:
display(top_action)
display(top_animation)
display(top_art)
display(top_classics)
display(top_comedy)
display(top_doc)
display(top_drama)
display(top_horror)
display(top_kids)
display(top_musical)
display(top_mystery)
display(top_romance)
display(top_science)
display(top_special)
display(top_sports)
display(top_tv)
display(top_west)

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
0,Action & Adventure,1.0,0.96,525,0.913371,Black Panther,2018,2010s
1,Action & Adventure,2.0,0.94,547,0.954802,Avengers: Endgame,2019,2010s
2,Action & Adventure,3.0,0.97,437,0.747646,Mission: Impossible - Fallout,2018,2010s
3,Action & Adventure,4.0,0.97,434,0.741996,Mad Max: Fury Road,2015,2010s
4,Action & Adventure,5.0,0.97,393,0.664783,Spider-Man: Into the Spider-Verse,2018,2010s
5,Action & Adventure,6.0,0.93,472,0.813559,Wonder Woman,2017,2010s
6,Action & Adventure,7.0,0.94,424,0.723164,Logan,2017,2010s
7,Action & Adventure,8.0,0.97,354,0.591337,Coco,2017,2010s
8,Action & Adventure,9.0,0.92,463,0.796610,Dunkirk,2017,2010s
9,Action & Adventure,10.0,0.91,483,0.834275,Star Wars: The Last Jedi,2017,2010s


,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
100,Animation,1.0,0.97,453,0.777778,Toy Story 4,2019,2010s
101,Animation,2.0,0.97,393,0.664783,Spider-Man: Into the Spider-Verse,2018,2010s
102,Animation,3.0,0.98,379,0.638418,Inside Out,2015,2010s
103,Animation,4.0,0.97,354,0.591337,Coco,2017,2010s
104,Animation,5.0,0.98,54,0.026365,Snow White and the Seven Dwarfs,1937,1930s
105,Animation,6.0,0.95,344,0.572505,Soul,2020,2020s
106,Animation,7.0,0.93,387,0.653484,Incredibles 2,2018,2010s
107,Animation,8.0,0.98,298,0.485876,Zootopia,2016,2010s
108,Animation,9.0,0.99,247,0.389831,Paddington 2,2018,2010s
109,Animation,10.0,0.98,298,0.485876,Up,2009,2000s


,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
200,Art House & International,1.0,0.98,464,0.798493,Parasite,2019,2010s
201,Art House & International,2.0,0.99,68,0.052731,The Cabinet of Dr. Caligari,1920,1920s
202,Art House & International,3.0,1.00,88,0.090395,Seven Samurai,1956,1950s
203,Art House & International,4.0,0.97,68,0.052731,La Grande illusion,1938,1930s
204,Art House & International,5.0,0.97,135,0.178908,Metropolis,1927,1920s
205,Art House & International,6.0,0.97,69,0.054614,"Nosferatu, a Symphony of Horror",1922,1920s
206,Art House & International,7.0,0.99,89,0.092279,The Battle of Algiers,1967,1960s
207,Art House & International,8.0,0.99,227,0.352166,Shoplifters,2018,2010s
208,Art House & International,9.0,1.00,60,0.037665,M,1931,1930s
209,Art House & International,10.0,1.00,49,0.016949,Battleship Potemkin,1925,1920s


,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
300,Classics,1.0,0.99,98,0.109228,It Happened One Night,1934,1930s
301,Classics,2.0,0.99,121,0.152542,Citizen Kane,1941,1940s
302,Classics,3.0,0.98,160,0.225989,The Wizard of Oz,1939,1930s
303,Classics,4.0,0.98,109,0.129944,Modern Times,1936,1930s
304,Classics,5.0,0.99,124,0.158192,Casablanca,1942,1940s
305,Classics,6.0,0.98,110,0.131827,Sunset Boulevard,1950,1950s
306,Classics,7.0,0.99,103,0.118644,All About Eve,1950,1950s
307,Classics,8.0,0.99,68,0.052731,The Cabinet of Dr. Caligari,1920,1920s
308,Classics,9.0,1.00,102,0.116761,The Philadelphia Story,1940,1940s
309,Classics,10.0,1.00,95,0.103578,Rebecca,1940,1940s


,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
500,Documentary,1.0,0.97,255,0.404896,Won't You Be My Neighbor?,2018,2010s
501,Documentary,2.0,0.99,210,0.320151,I Am Not Your Negro,2017,2010s
502,Documentary,3.0,0.99,186,0.274953,Apollo 11,2019,2010s
503,Documentary,4.0,0.99,208,0.316384,Summer of Soul,2021,2020s
504,Documentary,5.0,0.98,203,0.306968,Life Itself,2014,2010s
505,Documentary,6.0,1.00,159,0.224105,Man on Wire,2008,2000s
506,Documentary,7.0,0.99,160,0.225989,Amazing Grace,2019,2010s
507,Documentary,8.0,1.00,123,0.156309,Minding the Gap,2018,2010s
508,Documentary,9.0,0.99,142,0.192090,Faces Places,2017,2010s
509,Documentary,10.0,0.99,130,0.169492,Collective,2020,2020s


...

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
1200,Science Fiction & Fantasy,1.0,0.98,160,0.225989,The Wizard of Oz,1939,1930s
1201,Science Fiction & Fantasy,2.0,0.96,525,0.913371,Black Panther,2018,2010s
1202,Science Fiction & Fantasy,3.0,0.94,547,0.954802,Avengers: Endgame,2019,2010s
1203,Science Fiction & Fantasy,4.0,0.97,453,0.777778,Toy Story 4,2019,2010s
1204,Science Fiction & Fantasy,5.0,0.97,434,0.741996,Mad Max: Fury Road,2015,2010s
1205,Science Fiction & Fantasy,6.0,0.97,393,0.664783,Spider-Man: Into the Spider-Verse,2018,2010s
1206,Science Fiction & Fantasy,7.0,0.93,472,0.813559,Wonder Woman,2017,2010s
1207,Science Fiction & Fantasy,8.0,0.94,424,0.723164,Logan,2017,2010s
1208,Science Fiction & Fantasy,9.0,0.91,483,0.834275,Star Wars: The Last Jedi,2017,2010s
1209,Science Fiction & Fantasy,10.0,0.92,460,0.790960,The Shape of Water,2017,2010s


,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
1300,Special Interest,1.0,0.98,47,0.013183,The Bride of Frankenstein,1935,1930s
1301,Special Interest,2.0,1.00,52,0.022599,Frankenstein,1931,1930s
1302,Special Interest,3.0,0.98,203,0.306968,Life Itself,2014,2010s
1303,Special Interest,4.0,1.00,159,0.224105,Man on Wire,2008,2000s
1304,Special Interest,5.0,0.94,283,0.457627,Her,2013,2010s
1305,Special Interest,6.0,0.98,53,0.024482,The Discreet Charm Of The Bourgeoisie,1972,1970s
1306,Special Interest,7.0,0.97,86,0.086629,Groundhog Day,1993,1990s
1307,Special Interest,8.0,0.95,225,0.348399,Amy,2015,2010s
1308,Special Interest,9.0,0.96,48,0.015066,Miracle on 34th Street,1947,1940s
1309,Special Interest,10.0,0.98,131,0.171375,Blackfish,2013,2010s


,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
1400,Sports & Fitness,1.0,0.98,141,0.190207,Murderball,2005,2000s
1401,Sports & Fitness,2.0,1.00,57,0.032015,Athlete A,2020,2020s
1402,Sports & Fitness,3.0,0.98,60,0.037665,Hoop Dreams,1994,1990s
1403,Sports & Fitness,4.0,0.98,49,0.016949,When We Were Kings,1996,1990s
1404,Sports & Fitness,5.0,0.97,102,0.116761,The King of Kong: A Fistful of Quarters,2007,2000s
1405,Sports & Fitness,6.0,0.96,102,0.116761,Undefeated,2012,2010s
1406,Sports & Fitness,7.0,0.91,254,0.403013,The Fighter,2010,2010s
1407,Sports & Fitness,8.0,0.98,46,0.011299,Surfwise,2007,2000s
1408,Sports & Fitness,9.0,0.97,65,0.047081,The Life and Times of Hank Greenberg,2000,2000s
1409,Sports & Fitness,10.0,0.91,69,0.054614,Rocky,1976,1970s


,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
1469,Television,1.0,0.98,203,0.306968,Life Itself,2014,2010s
1470,Television,2.0,1.00,62,0.041431,Afghan Star,2009,2000s
1471,Television,3.0,0.98,65,0.047081,A Film Unfinished,2010,2010s
1472,Television,4.0,0.94,133,0.175141,Room 237,2013,2010s
1473,Television,5.0,0.97,65,0.047081,49 Up,2006,2000s
1474,Television,6.0,0.96,89,0.092279,The Return,2004,2000s
1475,Television,7.0,0.94,108,0.128060,Behind the Candelabra,2013,2010s
1476,Television,8.0,0.95,66,0.048964,"Not Quite Hollywood: The Wild, Untold Story of...",2008,2000s
1477,Television,9.0,0.95,78,0.071563,Being Elmo: A Puppeteer's Journey,2011,2010s
1478,Television,10.0,0.96,45,0.009416,Los Angeles Plays Itself,2003,2000s


,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
1526,Western,1.0,1.00,52,0.022599,The Treasure of the Sierra Madre,1948,1940s
1527,Western,2.0,0.97,77,0.069680,"The Good, the Bad and the Ugly",1966,1960s
1528,Western,3.0,1.00,45,0.009416,Stagecoach,1939,1930s
1529,Western,4.0,0.95,60,0.037665,High Noon,1952,1950s
1530,Western,5.0,0.96,106,0.124294,Unforgiven,1992,1990s
1531,Western,6.0,0.95,275,0.442561,True Grit,2010,2010s
1532,Western,7.0,0.98,51,0.020716,A Fistful of Dollars,1964,1960s
1533,Western,8.0,0.98,43,0.005650,Rio Bravo,1959,1950s
1534,Western,9.0,0.96,50,0.018832,The Searchers,1956,1950s
1535,Western,10.0,0.95,65,0.047081,Once Upon a Time in the West,1968,1960s


In [644]:
frames = [top_action, top_animation, top_art, top_classics, top_comedy, top_doc, top_drama, top_horror, top_kids, top_musical, top_mystery, top_romance, top_science, top_special, top_sports, top_tv, top_west]

In [645]:
top_result = pd.concat(frames)
top_result

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
0,Action & Adventure,1.0,0.96,525,0.913371,Black Panther,2018,2010s
1,Action & Adventure,2.0,0.94,547,0.954802,Avengers: Endgame,2019,2010s
2,Action & Adventure,3.0,0.97,437,0.747646,Mission: Impossible - Fallout,2018,2010s
3,Action & Adventure,4.0,0.97,434,0.741996,Mad Max: Fury Road,2015,2010s
4,Action & Adventure,5.0,0.97,393,0.664783,Spider-Man: Into the Spider-Verse,2018,2010s
...,...,...,...,...,...,...,...,...
1531,Western,6.0,0.95,275,0.442561,True Grit,2010,2010s
1532,Western,7.0,0.98,51,0.020716,A Fistful of Dollars,1964,1960s
1533,Western,8.0,0.98,43,0.005650,Rio Bravo,1959,1950s
1534,Western,9.0,0.96,50,0.018832,The Searchers,1956,1950s


In [646]:
fig17 = px.bar(top_result, x="Rank", y="Scaled Reviews", color="Decade")
fig17.show()

# 4. The favorite movie overall 
According to "Rotten Tomatoes", the favorite movie overall is "US(2019)", Horror Genre!

In [647]:
first = top_result.loc[top_result["Rank"] == 1]
first

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
0,Action & Adventure,1.0,0.96,525,0.913371,Black Panther,2018,2010s
100,Animation,1.0,0.97,453,0.777778,Toy Story 4,2019,2010s
200,Art House & International,1.0,0.98,464,0.798493,Parasite,2019,2010s
300,Classics,1.0,0.99,98,0.109228,It Happened One Night,1934,1930s
500,Documentary,1.0,0.97,255,0.404896,Won't You Be My Neighbor?,2018,2010s
500,Documentary,1.0,0.97,255,0.404896,Won't You Be My Neighbor?,2018,2010s
600,Drama,1.0,0.99,121,0.152542,Citizen Kane,1941,1940s
700,Horror,1.0,0.93,553,0.966102,Us,2019,2010s
800,Kids & Family,1.0,0.98,160,0.225989,The Wizard of Oz,1939,1930s
900,Musical & Performing Arts,1.0,0.98,160,0.225989,The Wizard of Oz,1939,1930s


In [648]:
first.sort_values(by=["Scaled Reviews"], ascending=False)

,Genre,Rank,RatingTomatometer,No. of Reviews,Scaled Reviews,Movie Title,YearB,Decade
700,Horror,1.0,0.93,553,0.966102,Us,2019,2010s
0,Action & Adventure,1.0,0.96,525,0.913371,Black Panther,2018,2010s
200,Art House & International,1.0,0.98,464,0.798493,Parasite,2019,2010s
100,Animation,1.0,0.97,453,0.777778,Toy Story 4,2019,2010s
500,Documentary,1.0,0.97,255,0.404896,Won't You Be My Neighbor?,2018,2010s
500,Documentary,1.0,0.97,255,0.404896,Won't You Be My Neighbor?,2018,2010s
1469,Television,1.0,0.98,203,0.306968,Life Itself,2014,2010s
900,Musical & Performing Arts,1.0,0.98,160,0.225989,The Wizard of Oz,1939,1930s
1200,Science Fiction & Fantasy,1.0,0.98,160,0.225989,The Wizard of Oz,1939,1930s
800,Kids & Family,1.0,0.98,160,0.225989,The Wizard of Oz,1939,1930s


# Future Projects 
Connected to this project, I would like to explore some of this data further. It will require me to scrape some additional websites and track down a little more information than the current parameters of this project needs. Here is the list of information I would like to gather. 
1. Audience ratings 
2. The money spent on production compared to money earned 
3. The money spent on advertising compared to money earned 
4. The money spent on production compared to audience ratings 